### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [35]:
# Calculate the number of unique schools
school_count = len(school_data_complete["school_name"].unique())

# Calculate the number of unique schools
student_count = len(school_data_complete["student_name"])

# Calculate the total budget for all schools/students
total_budget = school_data["budget"].sum()

# Calculate the average math score
avg_math = school_data_complete["math_score"].mean()

# Calculate the average reading score
avg_reading = school_data_complete["reading_score"].mean()

#Calculate the percentage of students with a passing math score (70 or greater)
passing_math = school_data_complete.loc[(school_data_complete["math_score"] >= 70),["student_name"]]
num_passing_math = passing_math["student_name"].count()
percent_passing_math = num_passing_math/student_count

#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading = school_data_complete.loc[(school_data_complete["reading_score"] >= 70),["student_name"]]
num_passing_reading = passing_reading["student_name"].count()
percent_passing_reading = num_passing_reading/student_count

#Calculate the percentage of students who passed math and reading (% Overall Passing)
passing_both = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & 
                    (school_data_complete["reading_score"] >= 70),["student_name"]]
num_passing_both = passing_both["student_name"].count()
percent_passing_both = num_passing_both/student_count

#Create a dataframe to hold the above results
dist_summary_df = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": student_count,
                              "Total Budget": total_budget,
                              "Average Math Score": avg_math,
                              "Average Reading Score": avg_reading,
                              "% Passing Math": percent_passing_math,
                              "% Passing Reading": percent_passing_reading,
                              "% Overall Passing": percent_passing_both})

#Optional: give the displayed data cleaner formatting
dist_summary_df["Total Students"] = dist_summary_df["Total Students"].map("{:,.0f}".format)
dist_summary_df["Total Budget"] = dist_summary_df["Total Budget"].map("${:,.2f}".format)
dist_summary_df["% Passing Math"] = (dist_summary_df["% Passing Math"]*100).map("{:.6f}%".format)
dist_summary_df["% Passing Reading"] = (dist_summary_df["% Passing Reading"]*100).map("{:.6f}%".format)
dist_summary_df["% Overall Passing"] = (dist_summary_df["% Overall Passing"]*100).map("{:.6f}%".format)
dist_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853%,85.805463%,65.172326%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [36]:
# Gather School Name, school type, school budget, total students
school_details = school_data[["school_name", "type","size","budget"]]


#Per Student Budget - calculation
school_details["Per Student Budget"] = school_details["budget"] / school_details["size"]

#Average Math and Reading Score using groupby
Avg_Scores = school_data_complete.groupby(['school_name'])[['math_score', 'reading_score']].mean()
school_avg = pd.merge(school_details, Avg_Scores, how="left", on=["school_name", "school_name"])

#% Passing Math
passing_math = school_data_complete.loc[(school_data_complete["math_score"] >= 70),["school_name","student_name"]]
num_passing_math  = passing_math.groupby(["school_name"]).count()
num_rename = num_passing_math.rename(columns = {"student_name":"Number Passing Math"})
school_math_num = pd.merge(school_avg, num_rename, how="left", on=["school_name", "school_name"])
school_math_num["% Passing Math"] = school_math_num["Number Passing Math"]/school_math_num["size"] *100

#% Passing Reading
passing_reading = school_data_complete.loc[(school_data_complete["reading_score"] >= 70),["school_name","student_name"]]
num_passing_reading = passing_reading.groupby(["school_name"]).count()
num_rename_reading = num_passing_reading.rename(columns = {"student_name":"Number Passing reading"})
school_reading_num = pd.merge(school_math_num, num_rename_reading, how="left", on=["school_name", "school_name"])
school_reading_num["% Passing Reading"] = school_reading_num["Number Passing reading"]/school_reading_num["size"] *100

#% Overall Passing (The percentage of students that passed math and reading.)
passing_both = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & 
                    (school_data_complete["reading_score"] >= 70),["school_name","student_name"]]
num_passing_both  = passing_both.groupby(["school_name"]).count()
num_rename_both = num_passing_both.rename(columns = {"student_name":"Number Passing both"})
school_both_num = pd.merge(school_reading_num, num_rename_both, how="left", on=["school_name", "school_name"])
school_both_num["% Overall Passing"] = school_both_num["Number Passing both"]/school_both_num["size"] *100

#Create a dataframe to hold the above results
school_summary_df = pd.DataFrame({"school_name": school_both_num["school_name"],
                                  "School Type": school_both_num["type"],
                                  "Total Students": school_both_num["size"],
                                  "Total School Budget": school_both_num["budget"],
                                  "Per Student Budget": school_both_num["Per Student Budget"],
                                  "Average Math Score": school_both_num["math_score"],
                                  "Average Reading Score": school_both_num["reading_score"],
                                  "% Passing Math": school_both_num["% Passing Math"],
                                  "% Passing Reading": school_both_num["% Passing Reading"],
                               "% Overall Passing": school_both_num["% Overall Passing"]})

#set school_name as index
school_summary_df = school_summary_df.set_index(['school_name'])

#sort the dataframe by school_name
school_summary_df = school_summary_df.sort_values(by=['school_name'])

#display the dataframe
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [37]:
#sort the school summary table descending by % overall passing
school_summary_df = school_summary_df.sort_values(by=['% Overall Passing'], ascending=False)

#Dislpay the top 5 records - these are the 5 highest performing
school_summary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [38]:
#sort the school summary table ascending by % overall passing
school_summary_df = school_summary_df.sort_values(by=['% Overall Passing'])

#Dislpay the top 5 records - these are the 5 lowest performing
school_summary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [39]:
#Filter for 9th grade students and average the score
grade_nine_math = school_data_complete.loc[(school_data_complete["grade"] == "9th"),["school_name","math_score"]]
grade_nine_math = grade_nine_math.rename(columns = {"math_score":"9th"}) 
grade_nine_math = grade_nine_math.groupby("school_name").mean()


#Filter for 10th grade students and average the score
grade_ten_math = school_data_complete.loc[(school_data_complete["grade"] == "10th"),["school_name","math_score"]]
grade_ten_math = grade_ten_math.rename(columns = {"math_score":"10th"}) 
grade_ten_math = grade_ten_math.groupby("school_name").mean()


#Filter for 11th grade students and average the score
grade_eleven_math = school_data_complete.loc[(school_data_complete["grade"] == "11th"),["school_name","math_score"]]
grade_eleven_math = grade_eleven_math.rename(columns = {"math_score":"11th"})
grade_eleven_math = grade_eleven_math.groupby("school_name").mean()

#Filter for 12th grade students and average the score
grade_twelve_math = school_data_complete.loc[(school_data_complete["grade"] == "12th"),["school_name","math_score"]]
grade_twelve_math = grade_twelve_math.rename(columns = {"math_score":"12th"}) 
grade_twelve_math = grade_twelve_math.groupby("school_name").mean()

#Merge the by grade dataframes into one
math_by_grade_ten = pd.merge(grade_nine_math, grade_ten_math, how="left", on=["school_name"])
math_by_grade_eleven = pd.merge(math_by_grade_ten, grade_eleven_math, how="left", on=["school_name"])
math_by_grade = pd.merge(math_by_grade_eleven, grade_twelve_math, how="left", on=["school_name"])

math_by_grade



,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [40]:
#Filter for 9th grade students and average the score
grade_nine_reading = school_data_complete.loc[(school_data_complete["grade"] == "9th"),["school_name","reading_score"]]
grade_nine_reading = grade_nine_reading.rename(columns = {"reading_score":"9th"}) 
grade_nine_reading = grade_nine_reading.groupby("school_name").mean()


#Filter for 10th grade students and average the score
grade_ten_reading = school_data_complete.loc[(school_data_complete["grade"] == "10th"),["school_name","reading_score"]]
grade_ten_reading = grade_ten_reading.rename(columns = {"reading_score":"10th"}) 
grade_ten_reading = grade_ten_reading.groupby("school_name").mean()


#Filter for 11th grade students and average the score
grade_eleven_reading = school_data_complete.loc[(school_data_complete["grade"] == "11th"),["school_name","reading_score"]]
grade_eleven_reading = grade_eleven_reading.rename(columns = {"reading_score":"11th"})
grade_eleven_reading = grade_eleven_reading.groupby("school_name").mean()

#Filter for 12th grade students and average the score
grade_twelve_reading = school_data_complete.loc[(school_data_complete["grade"] == "12th"),["school_name","reading_score"]]
grade_twelve_reading = grade_twelve_reading.rename(columns = {"reading_score":"12th"}) 
grade_twelve_reading = grade_twelve_reading.groupby("school_name").mean()

#Merge the by grade dataframes into one
reading_by_grade_ten = pd.merge(grade_nine_reading, grade_ten_reading, how="left", on=["school_name"])
reading_by_grade_eleven = pd.merge(reading_by_grade_ten, grade_eleven_reading, how="left", on=["school_name"])
reading_by_grade = pd.merge(reading_by_grade_eleven, grade_twelve_reading, how="left", on=["school_name"])

reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [41]:
#select the fields for breakdown of scores by school spending
score_spending = school_summary_df[["Per Student Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


# Bin Values  
bins = [0, 585, 630, 645, 680]

# Bin Names
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

#Cut the data by the bins and group to get the mean for each bin
score_spending["Per Student Budget"] = pd.cut(score_spending["Per Student Budget"], bins, labels=group_names, include_lowest=True)
score_spending_df = score_spending.groupby("Per Student Budget"). mean()
score_spending_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [42]:
#select the fields for breakdown of scores by school size
school_size = school_summary_df[["Total Students","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


# Bin Values  
bins = [0, 1000, 2000, 5000]

# Bin Names
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#Cut the data by the bins and group to get the mean for each bin
school_size["Total Students"] = pd.cut(school_size["Total Students"], bins, labels=group_names, include_lowest=True)
school_size_df = school_size.groupby("Total Students"). mean()
school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Total Students,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [43]:
#select the fields for breakdown of scores by school type
school_type = school_summary_df[["School Type","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#group the data to get the mean for school type
groupby_school_type = school_type.groupby(["School Type"]).mean()
groupby_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


# Analysis of PyCitySchools Data

### Noted Trends 

#### 1.  Higher per student spend does not produce higher test results.  It was shown in a couple of the tables that there is actually a correlation of more money being spent leading lower test scores.  The Scores by School Spending is the most obvious example, but if you look at the top 5 scoring schools and compare them the the bottom 5 scoring schools; this trend is evident in the numbers seen there as well.

#### 2. Smaller school size is indicative of higher test scores and passing rates.  It it appears that the schools with fewer students are doing better on the testing based on the Scores by School Size table and comparing the top 5 and bottom 5 schools.  This could be due to smaller class sizes and more individalized attention.

#### 3. The testing is consistent across the grades in each school showing that there is not a weakness in teachers in a specific area/grade.

#### 4. While it would be easy to assume that the Charter schools, which appear to have less funding and smaller sizes, are doing better only due to those reasons; there are factors not accounted for in this data.  Often charter schools do not have the support services in place (hence the lower funding) for students with learning disabilities.  This could be contributing to lower scores in the better funded schools with more students.